# Python - Consultas SQL a una BD MySQL

In [2]:
!pip install mysql-connector-python

     |████████████████████████████████| 30.9 MB 86 kB/s 


In [22]:
#importar libreria para trabjar con BD
import mysql.connector

In [23]:
import mysql.connector
from mysql.connector import errorcode

DB_NAME = 'employees'

TABLES = {}
TABLES['employees'] = (
    "CREATE TABLE `employees` ("
    "  `emp_no` int(11) NOT NULL AUTO_INCREMENT,"
    "  `birth_date` date NOT NULL,"
    "  `first_name` varchar(14) NOT NULL,"
    "  `last_name` varchar(16) NOT NULL,"
    "  `gender` enum('M','F') NOT NULL,"
    "  `hire_date` date NOT NULL,"
    "  PRIMARY KEY (`emp_no`)"
    ") ENGINE=InnoDB")

TABLES['salaries'] = (
    "CREATE TABLE `salaries` ("
    "  `emp_no` int(11) NOT NULL,"
    "  `salary` int(11) NOT NULL,"
    "  `from_date` date NOT NULL,"
    "  `to_date` date NOT NULL,"
    "  PRIMARY KEY (`emp_no`,`from_date`), KEY `emp_no` (`emp_no`),"
    "  CONSTRAINT `salaries_ibfk_1` FOREIGN KEY (`emp_no`) "
    "     REFERENCES `employees` (`emp_no`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

In [24]:
config = {
  'user': 'admin',
  'password': 'utec2021',
  'host': 'database-2.c4w6fd2h317c.us-east-1.rds.amazonaws.com',
  'database': 'employees',
  'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

In [25]:
def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(DB_NAME))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(DB_NAME))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        print("Database {} created successfully.".format(DB_NAME))
        cnx.database = DB_NAME
    else:
        print(err)
        exit(1)

In [15]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table employees: already exists.
Creating table departments: already exists.
Creating table salaries: already exists.
Creating table dept_emp: already exists.
Creating table dept_manager: already exists.
Creating table titles: already exists.


In [26]:
from datetime import date, datetime, timedelta
import mysql.connector

cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

tomorrow = datetime.now().date() + timedelta(days=1)

add_employee = ("INSERT INTO employees "
               "(first_name, last_name, hire_date, gender, birth_date) "
               "VALUES (%s, %s, %s, %s, %s)")
add_salary = ("INSERT INTO salaries "
              "(emp_no, salary, from_date, to_date) "
              "VALUES (%(emp_no)s, %(salary)s, %(from_date)s, %(to_date)s)")

data_employee = ('Diego', 'Benavides', tomorrow, 'M', date(1977, 6, 14))

# Insert new employee
cursor.execute(add_employee, data_employee)
emp_no = cursor.lastrowid

# Insert salary information
data_salary = {
  'emp_no': emp_no,
  'salary': 50000,
  'from_date': tomorrow,
  'to_date': date(9999, 1, 1),
}
cursor.execute(add_salary, data_salary)

# Make sure data is committed to the database
cnx.commit()

cursor.close()
cnx.close()

In [27]:
import datetime
import mysql.connector

cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

query = ("SELECT first_name, last_name, hire_date FROM employees ")
         #"WHERE hire_date BETWEEN %s AND %s")

hire_start = datetime.date(1999, 1, 1)
hire_end = datetime.date(1999, 12, 31)

#cursor.execute(query, (hire_start, hire_end))
cursor.execute(query)

for (first_name, last_name, hire_date) in cursor:
  print("{}, {} was hired on {:%d %b %Y}".format(
    last_name, first_name, hire_date))

cursor.close()
cnx.close()

Vanderkelen, Geert was hired on 07 Oct 2021
Benavides, Diego was hired on 07 Oct 2021


In [29]:
import pandas as pd

cnx = mysql.connector.connect(**config)
df = pd.read_sql("SELECT * FROM employees.employees", cnx)
df.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,1,1977-06-14,Geert,Vanderkelen,M,2021-10-07
1,2,1977-06-14,Diego,Benavides,M,2021-10-07


In [30]:
cursor.close()
cnx.close()

In [32]:
import pandas as pd

cnx = mysql.connector.connect(**config)
df = pd.read_sql("SELECT A.*, B.* FROM employees.employees A INNER JOIN employees.salaries B ON A.emp_no = B.emp_no", cnx)
df.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date,emp_no,salary,from_date,to_date
0,1,1977-06-14,Geert,Vanderkelen,M,2021-10-07,1,50000,2021-10-07,9999-01-01
1,2,1977-06-14,Diego,Benavides,M,2021-10-07,2,50000,2021-10-07,9999-01-01


# Cargar un archivo CSV a la base de datos

In [54]:
config = {
  'user': 'admin',
  'password': 'utec2021',
  'host': 'database-2.c4w6fd2h317c.us-east-1.rds.amazonaws.com',
  'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

In [55]:
def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format("sandbox"))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))

try:
    cursor.execute("USE {}".format("sandbox"))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format("sandbox"))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        print("Database {} created successfully.".format("sandbox"))
        cnx.database = DB_NAME
    else:
        print(err)

Database sandbox does not exists.
Failed creating database: 3719: 'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.
Database sandbox created successfully.


In [56]:
cursor.close()
cnx.close()

In [39]:
import pandas as pd
df = pd.read_csv('dataset/houseprices.csv', index_col=False, delimiter = ';')
df.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,2/05/14 00:00,313000.0,3,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,2/05/14 00:00,2384000.0,5,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119,USA
2,2/05/14 00:00,342000.0,3,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
3,2/05/14 00:00,420000.0,3,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008,USA
4,2/05/14 00:00,550000.0,4,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052,USA


In [57]:
import datetime
import mysql.connector

config = {
  'user': 'admin',
  'password': 'utec2021',
  'host': 'database-2.c4w6fd2h317c.us-east-1.rds.amazonaws.com',
  'database': 'sandbox',
  'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

In [58]:
TABLES['houseprices'] = (
    "CREATE TABLE `houseprices` ("
    "  `date` date,"
    "  `price` double,"
    "  `bedrooms` int,"
    "  `bathrooms` double,"
    "  `sqft_living` int,"
    "  `sqft_lot` int,"
    "  `floors` double,"
    "  `waterfront` int,"
    "  `view` int,"
    "  `condition` int,"
    "  `sqft_above` int,"
    "  `sqft_basement` int,"
    "  `yr_built` int,"
    "  `yr_renovated` int,"
    "  `street` varchar(255),"
    "  `city` varchar(50),"
    "  `statezip` varchar(50),"
    "  `country` varchar(50)"
    ") ENGINE=InnoDB")
cursor.execute(TABLES['houseprices'])
cursor.close()
cnx.close()

In [59]:
cursor.close()
cnx.close()

In [61]:
import datetime
import mysql.connector

config = {
  'user': 'admin',
  'password': 'utec2021',
  'host': 'database-2.c4w6fd2h317c.us-east-1.rds.amazonaws.com',
  'database': 'sandbox',
  'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

In [63]:
for i,row in df.iterrows():
    #aqui %S significa valores string/cadenas 
    sql = "INSERT INTO sandbox.houseprices VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    cursor.execute(sql, tuple(row))
    print("Record inserted")
    # la conexión no es auto enviada por defecto, por ese motivo debemos enviarla para guardar los cambios
    cnx.commit()

cursor.close()
cnx.close()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i